### This notebook does a bare bones implementation of RAG using the unprocessed MIDS intranet content as is, no tuning/tweaking. OpenAI gpt 3.5 is used as the llm

In [1]:
import os
import hashlib
from langchain_community.vectorstores import Qdrant
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, pipeline
from langchain.llms import HuggingFacePipeline

/home/reson/miniconda3/envs/LangChain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_embeddings = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.6.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





#### Load data

In [3]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [4]:
def create_document(file_path, university):
    content = read_file(file_path)
    file_name = os.path.basename(file_path)
    source_url = file_name.replace('.txt', '').replace('_', '/')
    metadata = {"source": source_url, "university": university}
    return Document(page_content=content, metadata=metadata)


In [5]:
def find_txt_files_recursive(directories):
    txt_files = []
    for directory in directories:
        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith('.txt'):
                    txt_files.append(os.path.join(root, file))
    return txt_files

In [6]:
import os

def find_txt_files(directories):
    txt_files = []
    for directory in directories:
        for file in os.listdir(directory):
            if os.path.isfile(os.path.join(directory, file)) and file.endswith('.txt'):
                txt_files.append(os.path.join(directory, file))
    return txt_files


In [7]:
data_directories = {
    'uc_berkeley_grad': '../data/university_data/uc_berkeley_grad',
    'csulb': '../data/university_data/csulb'
}

documents = []
for university, directory in data_directories.items():
    file_paths = find_txt_files([directory])
    documents.extend([create_document(file_path, university) for file_path in file_paths])


In [8]:
len(documents)

52

In [9]:
documents[0]

Document(page_content="\nWriting the Personal Statement - Berkeley Graduate Division\n\nSkip to content\n\n\t\t\t\t\t\t\t\t\tApply Now\t\t\t\t\t\t\t\t\n\nToggle Menu\n\nStudent Resources\n\nStaff Resources\n (opens in a new tab)\n\nNews & Events\n\n\t\t\t\t\t\t\t\t\tApply Now\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t\t\t\tToggle Search\t\t\t\t\t\t\t\t\t\t\t\n\nSearch for:\n\n\t\t\t\tSearch\t\t\t\n\nAbout\nAbout\nAbout\nThe Graduate Division serves more than 13,000 students in over 100 graduate degree programs. We are here to help you from the time you are admitted until you complete your graduate program.\n\nAbout the Graduate Division\n\nContact the Graduate Division\n\nWelcome from the Vice Provost and Dean\n\nDiversity Initiatives\n\nGraduate Diversity Task Force Recommendations\n\nAnti-Racism Initiatives\n\nLand Acknowledgement\n\nAdmissions\nAdmissions\nAdmissions\nWe're thrilled you're considering Berkeley for your graduate study. We offer more than 100 programs for master's, profess

#### Chunk documents based on semantic meaning

In [10]:
# text_splitter = SemanticChunker(
#     base_embeddings,
#     breakpoint_threshold_type='percentile'
# )

In [11]:
# split_documents = text_splitter.split_documents(documents)

In [13]:
# text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=0)
# split_documents = text_splitter_recursive.split_documents(documents)

In [10]:
# Use a simpler text splitter to reduce VRAM usage
text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)

def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

split_documents = split_documents_with_metadata(documents, text_splitter_recursive)


#### De-duplicate documents, this is a temporary workaround to strip the site header and footer text (since the data hasn't been sanitized yet)

In [11]:
# Example of text we're stripping out duplicate copies of
# This document doesnt actually tell us anything (and we have another copy of it for every site page)
split_documents[0]

Document(page_content="Writing the Personal Statement - Berkeley Graduate Division\n\nSkip to content\n\n\t\t\t\t\t\t\t\t\tApply Now\t\t\t\t\t\t\t\t\n\nToggle Menu\n\nStudent Resources\n\nStaff Resources\n (opens in a new tab)\n\nNews & Events\n\n\t\t\t\t\t\t\t\t\tApply Now\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t\t\t\tToggle Search\t\t\t\t\t\t\t\t\t\t\t\n\nSearch for:\n\n\t\t\t\tSearch\t\t\t\n\nAbout\nAbout\nAbout\nThe Graduate Division serves more than 13,000 students in over 100 graduate degree programs. We are here to help you from the time you are admitted until you complete your graduate program.\n\nAbout the Graduate Division\n\nContact the Graduate Division\n\nWelcome from the Vice Provost and Dean\n\nDiversity Initiatives\n\nGraduate Diversity Task Force Recommendations\n\nAnti-Racism Initiatives\n\nLand Acknowledgement\n\nAdmissions\nAdmissions\nAdmissions\nWe're thrilled you're considering Berkeley for your graduate study. We offer more than 100 programs for master's, professio

In [12]:
# Generates a hash of the document text, strips out duplicates
def deduplicate_documents(documents):
    seen_hashes = set()
    unique_documents = []
    for doc in documents:
        doc_hash = hashlib.md5(doc.page_content.encode('utf-8')).hexdigest()
        if doc_hash not in seen_hashes:
            seen_hashes.add(doc_hash)
            unique_documents.append(doc)
    return unique_documents

In [13]:
# Include university metadata in doc context

def format_docs(docs, max_length=20000):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"University: {doc.metadata['university']}\nSource: {doc.metadata['source']}\n\n{doc.page_content}")
    content = "\n\n".join(formatted_docs)
    tokens = tokenizer.tokenize(content)
    if len(tokens) > max_length:
        truncated_tokens = tokens[:max_length]
        return tokenizer.convert_tokens_to_string(truncated_tokens)
    return content


In [14]:
len(split_documents)

2384

In [15]:
split_documents = deduplicate_documents(split_documents)

In [16]:
len(split_documents)

1610

In [17]:
split_documents[0]

Document(page_content="Writing the Personal Statement - Berkeley Graduate Division\n\nSkip to content\n\n\t\t\t\t\t\t\t\t\tApply Now\t\t\t\t\t\t\t\t\n\nToggle Menu\n\nStudent Resources\n\nStaff Resources\n (opens in a new tab)\n\nNews & Events\n\n\t\t\t\t\t\t\t\t\tApply Now\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t\t\t\tToggle Search\t\t\t\t\t\t\t\t\t\t\t\n\nSearch for:\n\n\t\t\t\tSearch\t\t\t\n\nAbout\nAbout\nAbout\nThe Graduate Division serves more than 13,000 students in over 100 graduate degree programs. We are here to help you from the time you are admitted until you complete your graduate program.\n\nAbout the Graduate Division\n\nContact the Graduate Division\n\nWelcome from the Vice Provost and Dean\n\nDiversity Initiatives\n\nGraduate Diversity Task Force Recommendations\n\nAnti-Racism Initiatives\n\nLand Acknowledgement\n\nAdmissions\nAdmissions\nAdmissions\nWe're thrilled you're considering Berkeley for your graduate study. We offer more than 100 programs for master's, professio

#### Load documents into vector db

In [18]:
vectorstore = Qdrant.from_documents(split_documents,
    base_embeddings,
    location=":memory:",
)

In [19]:
retriever = vectorstore.as_retriever()

# Consumes ~ 3.8GB VRAM to documents into db (using semantic chunking)

In [21]:
!nvidia-smi

Sat Jun 15 16:41:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0  On |                  Off |
| 72%   52C    P0            368W /  450W |    3781MiB /  24564MiB |     59%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Prompt definitions

In [29]:
# We will need to tune this prompt

baseline_user_prompt = """
You are a helpful assistant that answers questions about university programs.
You will be provided a list of relevant context documents and a question.
Provide an answer to the question based on information from the relevant context

Please answer the question below only based on the context information provided.

### Here is a context:
{context} 

### Here is a question:
{question}
"""

In [30]:
rag_prompt = ChatPromptTemplate.from_template(baseline_user_prompt)

In [31]:
rag_prompt.invoke({
    "context": ["abcd", "efgh"],
    "question": "jklmnop?"
})

ChatPromptValue(messages=[HumanMessage(content="\nYou are a helpful assistant that answers questions about university programs.\nYou will be provided a list of relevant context documents and a question.\nProvide an answer to the question based on information from the relevant context\n\nPlease answer the question below only based on the context information provided.\n\n### Here is a context:\n['abcd', 'efgh'] \n\n### Here is a question:\njklmnop?\n")])

### Pipe retriever and llm together

#### Try with OpenAI GPT3.5 as baseline

##### Requires OPENAI_API_KEY env var to be set

In [22]:
os.environ['OPENAI_API_KEY'] = ""

In [65]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [66]:
rag_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough() }
    | rag_prompt
    | llm
)

In [45]:
rag_chain.invoke("What is the MIDS program").content

"The MIDS program is an online degree offered by UC Berkeley's School of Information that prepares data science professionals to solve real-world problems. It equips graduates with interdisciplinary skills to tackle complex human, social, economic, and health issues using data."

In [46]:
rag_chain.invoke("What are the prerequisites for taking the 210 course?").content

'The prerequisites for taking the Data Science 210A Capstone for Early Career Data Scientists course are DATASCI 200, DATASCI 201A, DATASCI 203, DATASCI 205, and DATASCI 207. Additionally, the course must be taken in the final term of the 5th Year MIDS program.'

In [47]:
rag_chain.invoke("Does the MIDS program have a payment plan? How does it work?").content

'Yes, the MIDS program offers a Fee Payment Plan (FPP) for the Fall and Spring terms. The FPP allows students to pay their tuition and fees in installments rather than a lump sum. More details and deadlines for the Fee Payment Plan can be found on the billing website. Please note that the Fee Payment Plan is not available for the Summer terms.'

In [49]:
rag_chain.invoke("What kind of additional fees does the program have outside of regular tuition? List the costs").content

'The UC Berkeley Masters of Information in Data Science (MIDS) program has the following additional fees outside of regular tuition:\n1. Berkeley Campus Fee:\n   - $790.25 for Fall 2023 & Spring 2024\n   - $431 for Summer 2024\n2. Document Management Fee: $107 (charged in the first term of enrollment)\n3. Instructional Resilience and Enhancement Fee:\n   - $117.50 for Fall and Spring terms\n   - $81 for Summer 2024 (if not enrolled in Spring 2024)\n4. UC Graduate and Professional Council (UCGPC) Fee: $3.50 for Fall and Spring terms\n5. Effective Fall 2024 Student Health Insurance Plan (SHIP): $3,078.00 for Fall and Spring terms only\n6. Immersion Fee: $500\n\nPlease note that fees are subject to change each academic year.'

In [52]:
rag_chain.invoke("Provide an short overview of the MIDS program degree requirements").content

'The Master of Information and Data Science (MIDS) program at UC Berkeley requires students to complete a total of 27 units, which includes core courses, elective courses, and a capstone project. Students must also attend an immersion program and fulfill all academic and administrative requirements set by the School of Information. Additionally, students are expected to maintain a minimum GPA of 3.0 in order to graduate from the program.'

In [54]:
rag_chain.invoke("What are the core courses students are required to take? Include the course numbers").content

'The core courses that students are required to take in the MIDS program are:\n\n1. Introduction to Data Science Programming (Course Number: Data Science 200)\n2. Research Design and Application for Data and Analysis (Course Number: Data Science 201)\n3. Statistics for Data Science (Course Number: Data Science 203)\n4. Fundamentals of Data Engineering (Course Number: Data Science 205)\n5. Applied Machine Learning (Course Number: Data Science 207)'

In [68]:
rag_chain.invoke("What is immersion? When does it happen and what are the requirements?").content

'Immersion is an academic event and a requirement for the UC Berkeley Masters of Information in Data Science (MIDS) program. It is an opportunity for students to meet faculty and peers in person on the UC Berkeley campus or in other relevant locations. Immersion typically lasts two to three days and includes presentations by faculty and industry experts, socializing with classmates, career advancement workshops, and networking opportunities. Immersion must be completed before the end of the Capstone course since participation is a graduation requirement.'

In [64]:
rag_chain.invoke("What are the main differences between 5th year MIDS students and regular MIDS students?").content

"The main differences between 5th Year MIDS students and regular MIDS students at UC Berkeley are as follows:\n\n1. **Streamlined Path for Cal Undergraduates:**\n   - The 5th Year MIDS program offers a streamlined path to the MIDS degree specifically for UC Berkeley undergraduates, allowing them to transition directly into the master's program.\n\n2. **Program Duration and Structure:**\n   - 5th Year MIDS is a lock-step program that requires students to complete the degree over four consecutive semesters (fall, spring, summer, and fall).\n   - Students in the 5th Year MIDS program are required to enroll in specific DATASCI courses each term in a predefined sequence.\n\n3. **Summer Practicum and Internship Encouragement:**\n   - 5th Year MIDS students are required to complete a 1-unit summer practicum course (DATASCI 293) and are strongly encouraged to pursue a full-time internship during the summer semester, although it is not mandatory.\n   - This focus on a summer practicum and inter

### Try with phi3 mini (with 4k context)

The 4k context only seems to be sufficent for more basic questions (heavy tuning on the chunking process might improve this and allow us to use this model instead)

In [23]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.34it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### The 4k context window is too small (atleast with semantic chunking) use the 128k model but cap it at 20k context

In [24]:
!nvidia-smi

Sun Jun  2 22:44:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0  On |                  Off |
|  0%   43C    P0             68W /  450W |    8929MiB /  24564MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
# Reduce context window to 20k
config = AutoConfig.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
config.max_position_embeddings = 20000  # Set to 20k tokens

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    max_length=20000
)

The repository for microsoft/Phi-3-mini-128k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-128k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.58it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
llm_phi3 = HuggingFacePipeline(pipeline=pipe)

In [23]:
baseline_phi3_user_prompt = """
<|user|>
You are a helpful assistant that answers questions about university programs.
You will be provided a list of relevant context documents and a question.
Provide an answer to the question based on information from the relevant context

Please answer the question below only based on the context information provided.

### Here is a context:
{context} 

### Here is a question:
{question}
<|end|>
<|assistant|>
"""

In [24]:
phi3_rag_prompt = ChatPromptTemplate.from_template(baseline_phi3_user_prompt)

In [25]:
# Given a query, retrieves documents
# Returns the number of tokens in the full context from the retrieved documents
def count_tokens_in_context(query):
    return len(tokenizer.tokenize("".join([doc.page_content for doc in retriever.invoke(query)])))

In [26]:
def extract_assistant_resp(raw_resp):
    if "<|assistant|>" not in raw_resp:
        raise f"Could not identify assistant token in response: {raw_resp}"
    
    return raw_resp.split("<|assistant|>")[1] 
    

In [27]:
phi3_rag_chain = (
    {"context": retriever | (lambda docs: format_docs(docs, max_length=20000)),
     "question": RunnablePassthrough() }
    | phi3_rag_prompt
    | llm_phi3
    | extract_assistant_resp
)

#### Q&A is weaker recursive text chunking

In [29]:
phi3_rag_chain.invoke("Does UC Berkeley offer programs in Data Science? What can you tell me about it?")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n Yes, UC Berkeley offers programs in Data Science. The university is renowned for its wide array of graduate programs, including over 120 graduate programs in 15 schools and colleges. These programs are designed to cater to the interests and career goals of master's, professional, and doctoral students. While the specific details about the Data Science program, such as the department or school it falls under, are not provided in the context, it is clear that UC Berkeley is committed to providing high-quality education and opportunities for students interested in fields like Data Science."

In [30]:
phi3_rag_chain.invoke("What are the admission requirements for the MIDS program at UC Berkeley")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'\n The admission requirements for the MIDS program at UC Berkeley, as per the provided context, include having a bachelor’s degree or recognized equivalent from an accredited institution, and a satisfactory scholastic average, usually a minimum grade-point average (GPA) of 3.0 (B) on a 4.0 scale. It is also recommended for applicants to have undergraduate research, training, or work experience. However, for specific requirements of the MIDS program, applicants should check with the program itself as they may have additional requirements not listed on this page.'

In [33]:
phi3_rag_chain.invoke("What kind of advanced courses does the MIDS program at UC Berkeley offer? Provide course names and an overview.")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n The Master of Information and Data Science (MIDS) program at UC Berkeley offers advanced courses designed to prepare students to work with diverse data sets and extract valuable insights using the latest analytical methods and tools. While specific course names are not listed in the provided context, the program's curriculum emphasizes the importance of inquiry and practical application in the field of information and data science."

In [34]:
phi3_rag_chain.invoke("Do both UC Berkeley and CSULB offer data science programs?")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'\n Based on the provided context, only CSULB offers a program in Engineering, which could potentially include data science. However, there is no explicit mention of a data science program at CSULB in the provided context. UC Berkeley is not mentioned in the context provided, so it cannot be determined if they offer a data science program from this information.'

In [35]:
phi3_rag_chain.invoke("What are the requirements to declare major as Computer Science for undergrad at CSULB?")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'\n To declare a major in Computer Science as an undergraduate at CSULB, students must first complete the General Education program, which includes at least 9 units in upper-division courses at CSULB. They must also meet precise major specific criteria, which typically involves completing certain lower division preparation courses. \n\nStudents who have not yet declared a major must do so by the time they reach 60 earned units. They must have a declared major by this time, whether they are Undeclared or in a pre-major status. \n\nAdditionally, students must possess a high school diploma or equivalent, not have been enrolled in college full-time for more than one term in the past five years, and maintain a 2.00 GPA or better in all college work attempted during that time. \n\nFinally, all undergraduate students, including those majoring in Computer Science, must demonstrate competency in writing skills as a requirement for graduation.'

In [36]:
phi3_rag_chain.invoke("What courses does the Computer science program at CSULB offer?")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n The provided context does not specify the courses offered in the Computer Science program at CSULB. It only lists various departments and programs such as Business Administration, Liberal Arts, Engineering, and more. For detailed course information, one might need to refer to the specific Computer Science department's catalog or contact the university directly."

In [34]:
phi3_rag_chain.invoke("What are the requirements for computer science at UC Berkeley?")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'\n For the 5th-Year MS in Electrical Engineering & Computer Sciences at UC Berkeley, which is designed for UC Berkeley students, there is no GRE requirement. Applicants are eligible if they have an undergraduate degree from an accredited institution or will have completed it before starting at Berkeley. A satisfactory GPA, typically a 3.0 (B) on a 4.0 scale, is recommended, and undergraduate research, training, or work experience can be beneficial.'

In [37]:
phi3_rag_chain.invoke("Provide an overview of course requirements for computer science at UC Berkeley")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n At UC Berkeley, the graduate program in Computer Science offers a Master's / Professional degree. The program does not require GRE scores for admission. The specific program of interest here is the 5th-Year MS program designed for UC Berkeley students. This program is available in the Fall term, with the application deadline set for December 11, 2023. While the context provided does not detail the course requirements for the program, it is understood that the program would typically include advanced coursework in electrical engineering and computer sciences, as indicated by the program's name."

In [39]:
phi3_rag_chain.invoke("Provide an overview of course requirements for computer science at CSULB")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n The provided context does not contain specific information about the course requirements for the Computer Science program at CSULB. It lists various departments and resources available at the university but does not detail the curriculum or prerequisites for any particular program. For detailed course requirements, one would need to refer directly to the Computer Science department's official catalog or contact the department directly."

In [ ]:
phi3_rag_chain.invoke("Provide an overview of course requirements for computer science at CSULB")

In [38]:
phi3_rag_chain.invoke("How does Electrical Engineering at UC Berkeley compare to CSULB?")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n I'm sorry, but based on the provided context, there is no information available comparing the Electrical Engineering program at UC Berkeley to the one at CSULB."

In [31]:
!nvidia-smi

Sat Jun 15 16:49:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0  On |                  Off |
| 62%   44C    P0             69W /  450W |   17453MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [45]:
!nvidia-smi

Sun Jun  2 22:53:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0  On |                  Off |
|  0%   43C    P8             30W /  450W |   22665MiB /  24564MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [48]:
phi3_rag_chain.invoke("Can I ")

Both `max_new_tokens` (=500) and `max_length`(=20000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'\n The estimated total tuition and fees for the MIDS program for a new student starting in Fall 2023 is currently $81,633. This cost is an estimate and most students complete the degree in approximately two years. However, the exact amount may vary based on several factors.'